In [1]:
import requests

In [9]:
%%time

results = {}
for i in range(40):
    resp = requests.get(f'https://jsonplaceholder.typicode.com/todos/{i}')
    results[i] = resp.json()

CPU times: user 1.13 s, sys: 176 ms, total: 1.31 s
Wall time: 16.2 s


In [10]:
results

{0: {},
 1: {'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False},
 2: {'userId': 1,
  'id': 2,
  'title': 'quis ut nam facilis et officia qui',
  'completed': False},
 3: {'userId': 1, 'id': 3, 'title': 'fugiat veniam minus', 'completed': False},
 4: {'userId': 1, 'id': 4, 'title': 'et porro tempora', 'completed': True},
 5: {'userId': 1,
  'id': 5,
  'title': 'laboriosam mollitia et enim quasi adipisci quia provident illum',
  'completed': False},
 6: {'userId': 1,
  'id': 6,
  'title': 'qui ullam ratione quibusdam voluptatem quia omnis',
  'completed': False},
 7: {'userId': 1,
  'id': 7,
  'title': 'illo expedita consequatur quia in',
  'completed': False},
 8: {'userId': 1,
  'id': 8,
  'title': 'quo adipisci enim quam ut ab',
  'completed': True},
 9: {'userId': 1,
  'id': 9,
  'title': 'molestiae perspiciatis ipsa',
  'completed': False},
 10: {'userId': 1,
  'id': 10,
  'title': 'illo est ratione doloremque quia maiores aut',
  'completed': True},
 11: {'use